# Fire Event Capture and Visualization using Sentinel-2 Imagery

This notebook processes wildfire data using Sentinel-2 imagery. It masks clouds, generates rectangular areas around fire points(~1km), and creates URLs for visualized images.

## Key Functions

- **`auth_engine_init(project_id)`**: Authenticates and initializes the Earth Engine.
- **`load_fire_data(csv_path)`**: Loads fire data from a CSV file.
- **`fetch_image_collection(longitude, latitude, start_date, end_date, buffer)`**: Fetches an image collection for a fire event.
- **`generate_image_url(collection, rectangle)`**: Generates a visualization URL for a clipped image.
- **`process_fire_events(fire_df)`**: Processes fire data and retrieves image URLs.
- **`download_images(img_urls)`**: Downloads images from generated URLs.

## Usage

1. Open Script In Colab: https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/scripts/image_extraction_script.ipynb

2. Set the project ID and CSV paths in the `main()` function.
3. Run `main()` to process the data and download images.


In [ ]:
import os
import ee
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def auth_earth_engine(project_id):
    """Authenticates and initializes Google Earth Engine."""
    ee.Authenticate()
    ee.Initialize(project=project_id)

def load_fire_data(csv_path):
    """Loads fire event data from a CSV file."""
    return pd.read_csv(csv_path)

def create_rectangle(longitude, latitude, buffer=0.02):
    """Creates a rectangle bounding box around the given coordinates."""
    return ee.Geometry.Rectangle([
        longitude - buffer,
        latitude - buffer,
        longitude + buffer,
        latitude + buffer
    ])

def fetch_image_collection(longitude, latitude, start_date, end_date, buffer=0.02):
    """Fetches Sentinel-2 images for the given location and date range."""
    rectangle = create_rectangle(longitude, latitude, buffer)
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(rectangle)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
                  .map(lambda img: img.divide(10000)))  # Normalize
    return collection, rectangle

def process_image(collection, rectangle):
    """Processes the image collection by selecting RGB bands and clipping."""
    if collection.size().getInfo() > 0:
        return collection.median().select(['B4', 'B3', 'B2']).clip(rectangle)
    return None

def generate_download_url(image, rectangle):
    """Generates a download URL for the processed image."""
    return image.getThumbURL({
        'min': 0, 'max': 0.5,
        'dimensions': 512,
        'region': rectangle,
        'format': 'png'
    })

def download_image(url, output_dir, filename):
    """Downloads an image from the given URL and saves it."""
    file_path = os.path.join(output_dir, filename)
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Downloaded: {file_path}")
    except requests.RequestException as e:
        print(f"Failed to download {filename}: {e}")

def process_fire_event(row, output_dir):
    """Processes a single fire event and downloads the corresponding image."""
    latitude, longitude = row['latitude'], row['longitude']
    acq_date = pd.to_datetime(row['acq_date'])

    start_date = (acq_date - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    end_date = (acq_date + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

    collection, rectangle = fetch_image_collection(longitude, latitude, start_date, end_date)
    image = process_image(collection, rectangle)

    if image:
        url = generate_download_url(image, rectangle)
        filename = f"{latitude}_{longitude}.png"
        download_image(url, output_dir, filename)

def process_fire_events_concurrently(fire_df, output_dir, max_workers=5):
    """Processes fire events concurrently using a thread pool."""
    os.makedirs(output_dir, exist_ok=True)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_fire_event, row, output_dir): row for _, row in fire_df.iterrows()}
        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing event: {e}")

def main():
    """Main function orchestrating the process."""
    project_id = 'wildfire-detection-ml'
    csv_path = '/Users/gunelaliyeva/PycharmProjects/wildfire-detection-using-satellite-imagery-main/data/raw/j2_viirs/2024_fire_nrt_J2V-C2_539017.csv'
    output_folder = 'img_folder'

    auth_earth_engine(project_id)
    fire_data = load_fire_data(csv_path)
    process_fire_events_concurrently(fire_data, output_folder, max_workers=5)

if __name__ == "__main__":
    main()



The command `!zip -r my_folder.zip my_folder` creates a compressed ZIP archive of the folder `my_folder`. This will be helpful for downloading the extracted images to your local machine in a single file.

In [ ]:
!zip -r MODIS_Images_2024.zip MODIS_Images_2024

In [ ]:
import shutil

# Specify the folder to remove
folder_to_remove = 'SUOMI_VIIRS_IMAGES_2023'

# Remove the folder and its contents
shutil.rmtree(folder_to_remove)
print(f"Folder '{folder_to_remove}' and its contents have been removed.")